# Artificial Neural Networks for Classification (binary)

## Import libraries

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

## Load data

In [2]:
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values
columns = dataset.columns[3:]

In [3]:
print(X.shape, y.shape)

(10000, 10) (10000,)


In [4]:
print(X[:5])
print(y[:5])

[[619 'France' 'Female' 42 2 0.0 1 1 1 101348.88]
 [608 'Spain' 'Female' 41 1 83807.86 1 0 1 112542.58]
 [502 'France' 'Female' 42 8 159660.8 3 1 0 113931.57]
 [699 'France' 'Female' 39 1 0.0 2 0 0 93826.63]
 [850 'Spain' 'Female' 43 2 125510.82 1 1 1 79084.1]]
[1 0 1 0 0]


## Preprocess data

### Encoding data

In [5]:
X[:5]

array([[619, 'France', 'Female', 42, 2, 0.0, 1, 1, 1, 101348.88],
       [608, 'Spain', 'Female', 41, 1, 83807.86, 1, 0, 1, 112542.58],
       [502, 'France', 'Female', 42, 8, 159660.8, 3, 1, 0, 113931.57],
       [699, 'France', 'Female', 39, 1, 0.0, 2, 0, 0, 93826.63],
       [850, 'Spain', 'Female', 43, 2, 125510.82, 1, 1, 1, 79084.1]],
      dtype=object)

In [6]:
le_X = LabelEncoder()
X[:, 2] = le_X.fit_transform(X[:, 2])
X[:5]

array([[619, 'France', 0, 42, 2, 0.0, 1, 1, 1, 101348.88],
       [608, 'Spain', 0, 41, 1, 83807.86, 1, 0, 1, 112542.58],
       [502, 'France', 0, 42, 8, 159660.8, 3, 1, 0, 113931.57],
       [699, 'France', 0, 39, 1, 0.0, 2, 0, 0, 93826.63],
       [850, 'Spain', 0, 43, 2, 125510.82, 1, 1, 1, 79084.1]],
      dtype=object)

In [7]:
ct = ColumnTransformer(
    transformers=[
        ('onehot', OneHotEncoder(drop='first'), [1])
    ], remainder='passthrough'
)
X = ct.fit_transform(X)
X[:5]

array([[0.0, 0.0, 619, 0, 42, 2, 0.0, 1, 1, 1, 101348.88],
       [0.0, 1.0, 608, 0, 41, 1, 83807.86, 1, 0, 1, 112542.58],
       [0.0, 0.0, 502, 0, 42, 8, 159660.8, 3, 1, 0, 113931.57],
       [0.0, 0.0, 699, 0, 39, 1, 0.0, 2, 0, 0, 93826.63],
       [0.0, 1.0, 850, 0, 43, 2, 125510.82, 1, 1, 1, 79084.1]],
      dtype=object)

In [8]:
le = LabelEncoder()
y = le.fit_transform(y)

In [9]:
print(list(X[0]))

[0.0, 0.0, 619, 0, 42, 2, 0.0, 1, 1, 1, 101348.88]


### Splitting and scaling data

In [10]:
X = X.astype(np.float32)

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
sc = StandardScaler()
X_train[:, 2:] = sc.fit_transform(X_train[:, 2:])
X_test[:, 2:] = sc.transform(X_test[:, 2:])

In [13]:
X_train

array([[ 0.        ,  0.        ,  0.3564997 , ...,  0.64920264,
         0.974817  ,  1.3676698 ],
       [ 1.        ,  0.        , -0.20389777, ...,  0.64920264,
         0.974817  ,  1.661254  ],
       [ 0.        ,  1.        , -0.96147215, ...,  0.64920264,
        -1.0258336 , -0.2528068 ],
       ...,
       [ 0.        ,  0.        ,  0.8650085 , ..., -1.5403509 ,
        -1.0258336 , -0.14276496],
       [ 0.        ,  0.        ,  0.15932281, ...,  0.64920264,
        -1.0258336 , -0.0508256 ],
       [ 1.        ,  0.        ,  0.47065476, ...,  0.64920264,
         0.974817  , -0.8145681 ]], dtype=float32)

## Building neural network

In [14]:
model = keras.Sequential([
    keras.layers.Dense(18, activation='relu', input_shape=(X_train.shape[1],)),
    keras.layers.Dense(12, activation='relu'),
    keras.layers.Dense(6, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

d:\Programs\Python\312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [15]:
list(X_train[:5])

[array([ 0.        ,  0.        ,  0.3564997 ,  0.9132475 , -0.65578586,
         0.34567967, -1.2184706 ,  0.80843616,  0.64920264,  0.974817  ,
         1.3676698 ], dtype=float32),
 array([ 1.        ,  0.        , -0.20389777,  0.9132475 ,  0.29493847,
        -0.34836912,  0.69683766,  0.80843616,  0.64920264,  0.974817  ,
         1.661254  ], dtype=float32),
 array([ 0.        ,  1.        , -0.96147215,  0.9132475 , -1.4163654 ,
        -0.6953935 ,  0.6186291 , -0.91668767,  0.64920264, -1.0258336 ,
        -0.2528068 ], dtype=float32),
 array([ 0.        ,  0.        , -0.9407167 , -1.0949934 , -1.1311481 ,
         1.3867528 ,  0.953212  , -0.91668767,  0.64920264, -1.0258336 ,
         0.9153928 ], dtype=float32),
 array([ 0.        ,  0.        , -1.3973368 ,  0.9132475 ,  1.6259525 ,
         1.3867528 ,  1.0574485 , -0.91668767, -1.5403509 , -1.0258336 ,
        -1.0596002 ], dtype=float32)]

## Teaching model

In [16]:
model.fit(X_train, y_train, epochs=100, batch_size=32)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 13s 7ms/step - accuracy: 0.7601 - loss: 0.5993
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.7999 - loss: 0.4428
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.8150 - loss: 0.4176
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.8419 - loss: 0.3860
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.8548 - loss: 0.3604
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.8571 - loss: 0.3525
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8652 - loss: 0.3329
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.8571 - loss: 0.3443
Epoch 9/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.8638 - loss: 0.3325
Epoch 10/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8582 - loss: 0.3353
Epoch 11/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.8635 - loss: 0.3316
Epoch 12/100
250/250 ━━━━━━━━━━━━━━━━━━━

## Testing

### Evaluation on Test data

In [17]:
y_pred = model.predict(X_test)
y_pred = (y_pred >= 0.5).astype(int)

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step


In [18]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.88      0.95      0.91      1607
           1       0.69      0.46      0.55       393

    accuracy                           0.85      2000
   macro avg       0.78      0.70      0.73      2000
weighted avg       0.84      0.85      0.84      2000



In [19]:
con_matrix = confusion_matrix(y_test, y_pred)
print(con_matrix)

[[1525   82]
 [ 214  179]]


In [20]:
acc = accuracy_score(y_test, y_pred)
print(f'Accuracy: {acc * 100:.2f}%')

Accuracy: 85.20%


### Evaluation of outside data

In [21]:
columns

Index(['CreditScore', 'Geography', 'Gender', 'Age', 'Tenure', 'Balance',
       'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary',
       'Exited'],
      dtype='object')

In [23]:
print(dataset.head(10).iloc[:, 3:])

   CreditScore Geography  Gender  Age  Tenure    Balance  NumOfProducts  \
0          619    France  Female   42       2       0.00              1   
1          608     Spain  Female   41       1   83807.86              1   
2          502    France  Female   42       8  159660.80              3   
3          699    France  Female   39       1       0.00              2   
4          850     Spain  Female   43       2  125510.82              1   
5          645     Spain    Male   44       8  113755.78              2   
6          822    France    Male   50       7       0.00              2   
7          376   Germany  Female   29       4  115046.74              4   
8          501    France    Male   44       4  142051.07              2   
9          684    France    Male   27       2  134603.88              1   

   HasCrCard  IsActiveMember  EstimatedSalary  Exited  
0          1               1        101348.88       1  
1          0               1        112542.58       0  
2     

In [24]:
data = {
    'Geography': ['France'],
    'CreditScore': [600],
    'Gender': ['Male'],
    'Age': [40],
    'Tenure': [3],
    'Balance': [60000],
    'NumOfProducts': [2],
    'HasCrCard': [1],
    'IsActiveMember': [1],
    'EstimatedSalary': [50000]
}

dataframe = pd.DataFrame(data, columns=columns)

In [26]:
print(dataframe)

   CreditScore Geography Gender  Age  Tenure  Balance  NumOfProducts  \
0          600    France   Male   40       3    60000              2   

   HasCrCard  IsActiveMember  EstimatedSalary Exited  
0          1               1            50000    NaN  


In [41]:
X_outside = dataframe.drop('Exited', axis=1).to_numpy()
X_outside[:, 2] = le_X.transform(X_outside[:, 2])
X_outside = ct.transform(X_outside)
X_outside[:, 2:] = sc.transform(X_outside[:, 2:])
X_outside = X_outside.astype(np.float32)

In [42]:
X_outside

array([[ 0.        ,  0.        , -0.5359852 ,  0.9132475 ,  0.10479359,
        -0.6953935 , -0.2578112 ,  0.80843616,  0.64920264,  0.974817  ,
        -0.8768322 ]], dtype=float32)

In [43]:
print(y_out_pred := model.predict(X_outside)[0, 0], y_out_pred >= 0.5)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
0.010588389 False
